In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
 os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
config_file = "configs/train_waymo_Copy1.yaml" #train_waymo_Copy1.yaml"#vgg_ssd300_voc0712_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-10 11:02:32,777 SSD INFO: Loaded configuration file configs/train_waymo_Copy1.yaml
2020-04-10 11:02:32,778 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet'   #vgg   #'basic'
        PRETRAINED: True
        OUT_CHANNELS: [512,1024,2048,512, 256, 128]    #[128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
        DEPTH: 50
INPUT:
    IMAGE_SIZE: 300
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    CHOICE: 'Adam'
    MAX_ITER: 65000
    LR_STEPS: [40000, 55000]
    GAMMA: 0.2
    BATCH_SIZE: 32
    LR: 1.5e-4
OUTPUT_DIR: 'outputs/res50_new_waymo_Adam_1_5e_f4' 
DATASET_DIR: "datasets"
2020-04-10 11:02:32,779 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 8
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: 300
  PIXEL_MEAN: [123, 117, 104]
LOG_STEP: 10
MODEL:
  BACKBONE:
    AFTER_TRAINED: False
    DEPTH: 50
    INPUT_CHANNELS:

In [ ]:
model = start_train(cfg)

resnet
Detector initialized. Total Number of params:  39.55M
Backbone number of parameters: 37.52M
SSD Head number of parameters: 2.03M
2020-04-10 11:02:37,047 SSD.trainer INFO: No checkpoint found.


In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)